In [102]:
import pandas as pd

# 读取数据

## 训练集&测试集处理

In [103]:
data_dir = 'data/data_tencent/'

In [104]:
# 特征数据集
df_user_feature = pd.read_csv(data_dir + 'userFeature.csv')
df_ad_feature = pd.read_csv(data_dir + 'adFeature.csv')
# 训练集
df_train = pd.read_csv(data_dir + 'train.csv')
# 测试集
df_test1 = pd.read_csv(data_dir + 'test1.csv')
df_test2 = pd.read_csv(data_dir + 'test2.csv')

In [105]:
df_user_feature.columns

Index(['LBS', 'age', 'appIdAction', 'appIdInstall', 'carrier',
       'consumptionAbility', 'ct', 'education', 'gender', 'house', 'interest1',
       'interest2', 'interest3', 'interest4', 'interest5', 'kw1', 'kw2', 'kw3',
       'marriageStatus', 'os', 'topic1', 'topic2', 'topic3', 'uid'],
      dtype='object')

In [106]:
df_user_feature.head()

,LBS,age,appIdAction,appIdInstall,carrier,consumptionAbility,ct,education,gender,house,...,interest5,kw1,kw2,kw3,marriageStatus,os,topic1,topic2,topic3,uid
0,950.0,4,NaN,NaN,1,2,3 1,7,2,NaN,...,52 100 72 131 116 11 71 12 8 113 28 73 6 132 9...,664359 276966 734911 103617 562294,11395 79112 115065 77033 36176,NaN,11,2,9826 105 8525 5488 7281,9708 5553 6745 7477 7150,NaN,26325489
1,803.0,2,NaN,NaN,1,1,3 1,2,1,NaN,...,NaN,338851 361151 542834 496283 229952,80263 39618 53539 180 38163,NaN,5 13,1,4391 9140 5669 1348 4388,9401 7724 1380 8890 7153,NaN,1184123
2,927.0,1,NaN,NaN,3,1,3 1,5,1,NaN,...,77 72 80 116 101 13 1 109 8 50 6 42 76 9 46 36...,746140 695808 126355 771775 411511,105115 71378 41409 74061 44005,NaN,13 10,1,1502 5488 9826 2187 8088,5005 9154 2756 5612 4209,NaN,76072711
3,486.0,4,NaN,NaN,3,1,1 3,7,1,NaN,...,100 80 92 37 116 13 47 4 71 8 50 28 98 115 6 4...,283399 402245 734509 654027 32061,74010 32918 67882 116802 20957,NaN,11,1 2,1619 7342 3064 9213 8525,810 2438 5659 1844 9262,NaN,63071413
4,112.0,5,NaN,NaN,3,2,3 1 4 2,6,1,NaN,...,131 116 13 8 6 132 42 9 59 18 58 64 129 103,562294 157603 589706 657719 495672,62764 97803 89066 55545 74061,NaN,11,1,477 9826 5808 644 2747,5483 2199 5424 1511 7751,NaN,81294159


In [107]:
df_ad_feature.head()

,aid,advertiserId,campaignId,creativeId,creativeSize,adCategoryId,productId,productType
0,177,8203,76104,1500666,59,282,0,6
1,2050,19441,178687,245165,53,1,0,6
2,1716,5552,158101,1080850,35,27,113,9
3,336,370,4833,119845,22,67,113,9
4,671,45705,352827,660519,42,67,0,4


In [108]:
df_train.columns

Index(['aid', 'uid', 'label'], dtype='object')

In [109]:
df_train.label.value_counts()

0    8376853
1     421961
Name: label, dtype: int64

In [110]:
# 插入test字段，0代表训练集，1代表测试集1，2代表测试集2
df_test1['test'] = 1
df_test2['test'] = 2

# 合并两组测试集合
df_test = pd.concat([df_test1, df_test2], axis=0, ignore_index=True)
df_test['label'] = -1

df_test.test.value_counts()

1    2265989
2    2265879
Name: test, dtype: int64

In [111]:
# 合并测试集和训练集
df_train_test = pd.concat([df_train, df_test], axis=0, ignore_index=True)
# 训练集的test列为0
df_train_test['test'].fillna(0, inplace=True)

df_train_test.label.value_counts()


 0    8376853
-1    4531868
 1     421961
Name: label, dtype: int64

## 关联特征数据

In [112]:
# 关联用户特征
df_data = pd.merge(df_train_test, df_user_feature, on=['uid'], how='left')
# 关联广告特征
df_data = pd.merge(df_data, df_ad_feature, on=['aid'], how='left')

In [113]:
df_data.isna().sum()

aid                          0
uid                          0
label                        0
test                         0
LBS                         88
age                          0
appIdAction           13128798
appIdInstall          13060063
carrier                      0
consumptionAbility           0
ct                           0
education                    0
gender                       0
house                 10917477
interest1              1209907
interest2              4488018
interest3             12977810
interest4             13128456
interest5              3349838
kw1                    1337508
kw2                     452732
kw3                   12719761
marriageStatus               1
os                           0
topic1                 1136438
topic2                  479002
topic3                12718078
advertiserId                 0
campaignId                   0
creativeId                   0
creativeSize                 0
adCategoryId                 0
productI

In [114]:
df_data.fillna(-1, inplace=True)
# df_data = pd.DataFrame(df_data.values, columns=df_data.columns)
df_data['label'] = df_data['label'].astype(float)  # 0&1: train; -1: test


In [115]:
df_data.to_csv('data/preprocess/df_train_test_merge.csv', index=False)

# 数据探索

In [116]:
df_data.head()

,aid,uid,label,test,LBS,age,appIdAction,appIdInstall,carrier,consumptionAbility,...,topic1,topic2,topic3,advertiserId,campaignId,creativeId,creativeSize,adCategoryId,productId,productType
0,699,78508957,0.0,0.0,576.0,1,-1,-1,0,1,...,4908 9745 7070 4203 3292,-1,-1,1082,295940,731679,59,13,0,6
1,1991,3637295,0.0,0.0,353.0,3,-1,-1,1,0,...,8445 6762 7608 523 7688,4518 862 3643 702 5247,-1,702,42104,1441131,53,10,4669,11
2,1119,19229018,0.0,0.0,94.0,4,-1,-1,0,0,...,941 4674 9735 8953 5352,-1,-1,3993,63752,798752,59,10,19256,11
3,2013,79277120,0.0,0.0,48.0,2,-1,-1,1,1,...,4538 6790 8850 3422 1940,2979 1877 358 666 4166,-1,6937,186348,1427984,35,89,3791,9
4,692,41528441,0.0,0.0,333.0,1,-1,-1,1,1,...,4535 5590 47 5570 2078,856 4587 1877 392 9103,-1,6946,296367,455396,59,24,3794,11


## 正负样本比例

In [117]:
df_data.label.value_counts()
# pos 421961 / neg 8376853

 0.0    8376853
-1.0    4531868
 1.0     421961
Name: label, dtype: int64

## 特征insight

In [118]:
df_data.columns

Index(['aid', 'uid', 'label', 'test', 'LBS', 'age', 'appIdAction',
       'appIdInstall', 'carrier', 'consumptionAbility', 'ct', 'education',
       'gender', 'house', 'interest1', 'interest2', 'interest3', 'interest4',
       'interest5', 'kw1', 'kw2', 'kw3', 'marriageStatus', 'os', 'topic1',
       'topic2', 'topic3', 'advertiserId', 'campaignId', 'creativeId',
       'creativeSize', 'adCategoryId', 'productId', 'productType'],
      dtype='object')

### aid
含义：广告id

In [120]:
df_data.aid.nunique()

173


### uid
- 含义：用户id


### label
- 含义：正负样本标记 train_pos 1 / train_neg 0 / test -1


### test
- 含义：测试数据集合标记，1表示测试集1，2表示测试集2

### LBS
- 含义：Location Based Services，相当于一个用户位置的分类


In [125]:
df_data.LBS.nunique()  # Location Based Services

864

### age
- 含义：年龄
- 已经是分类了，无缺失值

In [127]:
df_data.age.unique()

array([1, 3, 4, 2, 5, 0])

### appIdAction
- 含义：
- 序列特征，string形式，各元素用空格分隔，缺失值用-1表示

In [128]:
df_data.appIdAction.unique()

array([-1, '2183 4807 2911', '3269 562 4041 4973 1088 5266', ...,
       '1559 5606 27 1266 2283 5531 725 1968 2964 4106 2892 2755',
       '3287 4058 562 2168',
       '517 3292 2482 1584 4572 3269 872 4324 4114 3457 5983 2192'],
      dtype=object)

### appIdInstall
- 含义：
- 序列特征，string形式，各元素用空格分隔，缺失值用-1表示

In [130]:
df_data.appIdAction.unique()[:3]

array([-1, '2183 4807 2911', '3269 562 4041 4973 1088 5266'], dtype=object)

### carrier
- 含义：运营商
- 离散特征，无缺失情况

In [133]:
df_data.carrier.unique()

array([0, 1, 3, 2])

### consumptionAbility
- 含义：消费能力等级
- 离散特征，无缺失值

In [134]:
df_data.consumptionAbility.unique()

array([1, 0, 2])

### ct
- 含义：
- 序列特征，无缺失值，各元素用空格分隔，缺失值用-1表示

In [135]:
df_data.ct.unique()

array(['0', '3', '3 1', '1 3 4', '4 1 3', '3 1 2', '1 3', '3 4 1 2',
       '3 2 4 1', '3 1 4', '3 4 1', '1 4 3', '1', '1 3 2 4', '3 1 4 2',
       '1 3 2', '3 2 4', '2 3 1', '4 3 1', '3 4', '3 2 1 4', '2 1 3',
       '3 2', '3 4 2', '3 2 1', '1 2 3', '1 3 4 2', '1 4 3 2', '1 2 3 4',
       '4', '2 3 1 4', '3 1 2 4', '1 2', '2 3', '1 4', '1 4 2 3', '4 3',
       '1 2 4 3', '2 4 1 3', '2 1 3 4', '4 1 3 2', '4 3 2', '4 3 1 2',
       '4 1', '4 2 3 1', '2', '4 1 2 3', '2 4 3 1', '2 4 3', '2 3 4',
       '3 4 2 1', '2 4', '2 3 4 1', '2 1', '4 2 1 3', '2 1 4 3', '4 2 3',
       '1 4 2', '1 2 4', '4 2 1', '4 3 2 1', '4 1 2', '2 4 1', '4 2',
       '2 1 4'], dtype=object)

In [ ]:
'education',
'gender', 'house', 'interest1', 'interest2', 'interest3', 'interest4',
'interest5', 'kw1', 'kw2', 'kw3', 'marriageStatus', 'os', 'topic1',
'topic2', 'topic3', 'advertiserId', 'campaignId', 'creativeId',
'creativeSize', 'adCategoryId', 'productId', 'productType'

### education
- 含义：受教育等级
- 离散特征，无缺失值，

In [136]:
df_data.education.unique()

array([1, 7, 2, 6, 5, 0, 3, 4])

In [82]:
user_feature_lst = []
ad_feature_lst = []
train_lst = []
test_lst = []

train = df_data[df_data['test'] == 0][['aid','label']]  # 训练集的test列为0
test1_index = df_data[df_data['test'] == 1].index  # 测试集1
test2_index = df_data[df_data['test'] == 2].index  # 测试集2

In [83]:
# 插入字段n_parts数据集进行分块，训练集分成五块1、2、3、4、5，测试集1为6、测试集2为7
# 也就是test字段与n_parts字段都是为了区分数据块，n_parts对训练集进行了分块
n_parts = 5
index = []
for i in range(n_parts):
    index.append([])  # 标记5块idx
aid_lst = list(df_train['aid'].unique())[:1]

for aid in aid_lst:
    ad_piece = train[train['aid'] == aid]
    for k in [0, 1]:  # 0&1: train
        # 记录训练集中各个广告的正负样本idx
        idx_lst = list(ad_piece[ad_piece['label'] == k].sample(frac=1, random_state=10).index)
        cut = [0]
        
        # Wt's this???????
        for i in range(n_parts):
            cut.append(int((i + 1) * len(idx_lst) / n_parts) + 1)
        for j in range(n_parts):
            index[j].extend(idx_lst[cut[j]: cut[j+1]])

In [86]:
se = pd.Series()
for n in range(n_parts):
    se = se.append(pd.Series(n + 1, index=index[n]))
se = se.append(pd.Series(6, index=test1_index)) 
se = se.append(pd.Series(7, index=test2_index))

df_data.insert(0, 'n_parts', list(pd.Series(df_data.index).map(se).values))
df_data.head()

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  """Entry point for launching an IPython kernel.


,n_parts,aid,uid,label,test,LBS,age,appIdAction,appIdInstall,carrier,...,topic1,topic2,topic3,advertiserId,campaignId,creativeId,creativeSize,adCategoryId,productId,productType
0,1.0,699,78508957,0.0,0,576,1,-1,-1,0,...,4908 9745 7070 4203 3292,-1,-1,1082,295940,731679,59,13,0,6
1,NaN,1991,3637295,0.0,0,353,3,-1,-1,1,...,8445 6762 7608 523 7688,4518 862 3643 702 5247,-1,702,42104,1441131,53,10,4669,11
2,NaN,1119,19229018,0.0,0,94,4,-1,-1,0,...,941 4674 9735 8953 5352,-1,-1,3993,63752,798752,59,10,19256,11
3,NaN,2013,79277120,0.0,0,48,2,-1,-1,1,...,4538 6790 8850 3422 1940,2979 1877 358 666 4166,-1,6937,186348,1427984,35,89,3791,9
4,NaN,692,41528441,0.0,0,333,1,-1,-1,1,...,4535 5590 47 5570 2078,856 4587 1877 392 9103,-1,6946,296367,455396,59,24,3794,11


In [87]:

df_data.n_parts.value_counts()

6.0    2265989
7.0    2265879
1.0       9366
4.0       9364
3.0       9364
2.0       9364
5.0       9363
Name: n_parts, dtype: int64